# Convert data

Convert json data to txt format requested by Yolo. One txt file per image with the annotations of each object, each line corresponding to an object with the coordinates of the bounding box and the label of the object.

In [1]:
import json
import os

In [2]:
def count_element_per_label(json_dir):
    label_dict = {}
    
    # Iterate over all JSON files in the directory
    for json_file in os.listdir(json_dir):
        if json_file.endswith(".json"):
            with open(os.path.join(json_dir, json_file), 'r') as f:
                data = json.load(f)
                for obj in data.get('objects', []):
                    label = obj['label']
                    # Count the occurrences of each label
                    if label not in label_dict:
                        label_dict[label] = 1
                    else:
                        label_dict[label] += 1
    return label_dict
    

### min_occurrences

There are a large number of labels that appear only a few times in the entire dataset (less than 100 classes, for a total of 40,000 images). It is therefore possible to filter them with min_occurrences.

In [3]:
def generate_label_dict(json_dir, min_occurrences=500):
    # Count the occurrences of each label
    label_dict = count_element_per_label(json_dir)
    
    # Filter the labels with less than min_occurrences occurrences
    label_dict = {k: v for k, v in label_dict.items() if v >= min_occurrences}
    
    # Generate the label dictionary
    label_dict = {label: i for i, label in enumerate(label_dict.keys())}
    
    return label_dict

In [4]:
def convert_to_yolo(json_path, output_dir, label_dict):
    # Load the json file
    with open(json_path, 'r') as f:
        data = json.load(f)

    # Read the image width and height from the JSON
    image_width = data['width']
    image_height = data['height']

    # Initialize the list to store annotations
    annotations = []

    # Iterate over all objects in the JSON
    for obj in data['objects']:
        label = obj['label']
        if label not in label_dict:
            continue  # Ignore the label if not in the dictionary

        class_id = label_dict[label]
        bbox = obj['bbox']

        # Extract the coordinates
        xmin, ymin, xmax, ymax = bbox['xmin'], bbox['ymin'], bbox['xmax'], bbox['ymax']

        # Normalize the coordinates
        x_center = ((xmin + xmax) / 2) / image_width
        y_center = ((ymin + ymax) / 2) / image_height
        width = (xmax - xmin) / image_width
        height = (ymax - ymin) / image_height

        # Append the annotation in YOLO format
        annotations.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    # Save the annotations to a text file
    output_file = os.path.join(output_dir, os.path.splitext(os.path.basename(json_path))[0] + ".txt")
    with open(output_file, 'w') as f:
        f.write("\n".join(annotations))

In [5]:
def convert_all_to_yolo(json_directory, output_directory, label_dict):
    for json_file in os.listdir(json_directory):
        if json_file.endswith(".json"):
            json_path = os.path.join(json_directory, json_file)
            convert_to_yolo(json_path, output_directory, label_dict)

    print(f"Annotations sauvegardées dans : {output_directory}")

In [6]:
# Path, only thing to change if needed
json_directory = "../datasets/annotations/"
output_directory = "../datasets/annotations_yolo/"

In [7]:
# Generate the label dictionary
label_dict = generate_label_dict(json_directory, min_occurrences=500)

# save the label dictionary
with open("label_dict.json", "w") as f:
    json.dump(label_dict, f)

print("Dictionnaire généré :", label_dict)

Dictionnaire généré : {'other-sign': 0, 'regulatory--keep-right--g1': 1, 'warning--curve-left--g2': 2, 'regulatory--keep-right--g4': 3, 'warning--pedestrians-crossing--g4': 4, 'regulatory--yield--g1': 5, 'regulatory--one-way-straight--g1': 6, 'regulatory--no-entry--g1': 7, 'regulatory--no-stopping--g15': 8, 'information--pedestrians-crossing--g1': 9, 'information--parking--g1': 10, 'regulatory--stop--g1': 11, 'regulatory--maximum-speed-limit-30--g1': 12, 'regulatory--priority-road--g4': 13, 'complementary--chevron-left--g1': 14, 'regulatory--maximum-speed-limit-60--g1': 15, 'regulatory--maximum-speed-limit-40--g1': 16, 'complementary--chevron-right--g1': 17, 'regulatory--maximum-speed-limit-50--g1': 18, 'regulatory--no-parking--g5': 19, 'regulatory--no-parking--g2': 20, 'regulatory--no-parking--g1': 21, 'warning--road-bump--g2': 22}


In [8]:
# Create the output directory if it does not exist
os.makedirs(output_directory, exist_ok=True)

# Load the label dictionary
with open("label_dict.json", "r") as f:
    label_dict = json.load(f)

# Convert all JSON files to YOLO format
convert_all_to_yolo(json_directory, output_directory, label_dict)

Annotations sauvegardées dans : ../datasets/annotations_yolo/
